<a href="https://colab.research.google.com/github/minson18/PM2.5-Predict/blob/main/PM2.5_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Root Path
os.chdir('/content/drive/MyDrive/Data Mining/HW1')

Mounted at /content/drive


In [ ]:
TRAIN_PATH = "train.csv"
TEST_PATH = "test_X.csv"

In [ ]:
import numpy as np
import pandas as pd
import math
import csv
from sklearn.model_selection import train_test_split
import random

In [ ]:
# Clean data: drop unneeded columns, replace invalid as -1
data_df = pd.read_csv(TRAIN_PATH)
data_df.drop(['Location             ', 'Date          ', 'ItemName                 '], axis = 1, inplace = True)
for col in data_df.columns:
  data_df[col] = pd.to_numeric(data_df[col], downcast="float", errors='coerce').fillna(-1)
raw_data = data_df.to_numpy()

In [ ]:
# Transform data into forms that
# rows into continuous hours
# columns into air pollution indices
def transform(raw_data):
  days = raw_data.shape[0] // 18
  data = raw_data[0:18, :].T
  for i in range(1, days):
    b = raw_data[18*i:18*(i+1), :].T
    data = np.concatenate((data, b), axis=0)

  return data

In [ ]:
# Filled invalid element by the next hour's value
def clean(a, avg=False):
  for i in reversed(range(a.shape[0])):
    for j in range(a.shape[1]):
      if a[i][j] == -1:        
        if avg:
          a[i][j] = np.mean(a[:, j])
        else:
          a[i][j] = a[i+1][j]
  
  return a

In [ ]:
# Normalize data by columns
def normalize(a):
  std = np.std(a, axis=0, dtype=np.float64)
  mean = np.mean(a, axis=0, dtype=np.float64)
  return (a-mean) / std

In [ ]:
data = transform(raw_data)
data_avg = clean(data, avg=True)
data_next = clean(data)

In [ ]:
# Construct training data, as using last 9 hours' data 
# to predict 10'th hour PM2.5 
def train_data(data):
  X = []
  y = []
  for i in range(9, data.shape[0]):
    t = data[i-9:i, :]
    t = t.reshape(-1)
    X.append(t)
    y.append([data[i, 9]])

  X = np.array(X)
  y = np.array(y)
  print(X.shape)
  print(y.shape)
  X = normalize(X)
  X = np.concatenate([X , np.ones((X.shape[0],1))], axis = 1)

  return X, y

In [ ]:
X_avg, y = train_data(data_avg)
X_next, y = train_data(data_next)

(5751, 162)
(5751, 1)
(5751, 162)
(5751, 1)


In [ ]:
# Clean data: drop unneeded columns, replace invalid as -1
test_df = pd.read_csv(TEST_PATH, header = None)
indices = test_df[0].unique()
test_df.drop([0, 1], axis = 1, inplace = True)
for col in test_df.columns:
  test_df[col] = pd.to_numeric(test_df[col], downcast="float", errors='coerce').fillna(0)
raw_test = test_df.to_numpy()

In [ ]:
test = transform(raw_test)
test_avg = clean(test, avg=True)
test_next = clean(test)

In [ ]:
# Construct testing data, as using last 9 hours' data 
# to predict 10'th hour PM2.5 
def test_data(test):
  X_test = []
  for i in range(1, test.shape[0]//9+1):
    t = test[i*9-9:i*9, :]
    t = t.reshape(-1)
    X_test.append(t)

  X_test = np.array(X_test)
  print(X_test.shape)
  X_test = normalize(X_test)
  X_test = np.concatenate([X_test,np.ones(shape = (X_test.shape[0],1))] , axis = 1)

  return X_test

In [ ]:
X_test_avg = test_data(test_avg)
X_test_next = test_data(test_next)

(244, 162)
(244, 162)


In [ ]:
# Find features that have correlation coefficient 
# with target larger than threshold
def feature_select(X, X_test, threshold=0.7):
  high_corr = []
  for i in range(X.shape[1]):  
    corr = np.corrcoef(X[:, i], y.reshape(-1))[0][1]
    if abs(corr) > threshold:
      high_corr.append(i)

  return X[:, high_corr], X_test[:, high_corr]

In [ ]:
# find partial derivative of lossfunction
def partial_derivative(X_batch, y_batch, m_stat):

  y_pred = np.dot(X_batch, m_stat)
  n = len(y_batch)
  df_dm = (-2/n) * np.dot(X_batch.T, (y_batch - y_pred))
  df_dm = df_dm.reshape(len(df_dm), -1)
  
  return df_dm

In [ ]:
def MSE(X, y, m_stat):
  y_pred = np.dot(X, m_stat)
  #print(y_pred)
  mse = np.sum((y_pred - y)**2) / len(y)
  
  return mse

In [ ]:
def training(X, y, batch_size=64, lr=0.0006, epochs=3000, reg_para=1e-6, fudge_factor=0, if_print=True, random_state=None):
  """
  Using linear regression and AddaGrad as optimizer
  lr = learning rate
  reg_para = regularization parameter
  fudge_factor = a small number to counter numerical instabiltiy in AdaGrad
  """
  if random_state is not None:
    random.seed(random_state)
    
  for epoch in range(1, epochs+1):

    #random initialize weight
    if epoch == 1:
      m_stat = np.random.rand(X.shape[1],1)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    X = X[indices]
    y = y[indices]
    
    cumulative_derivative = np.zeros((X.shape[1],1)) #store comulative derivative
    for batch in range(len(X)//batch_size):
      start = batch*batch_size
      stop = (batch*batch_size) + batch_size

      X_batch = X[start:stop]
      y_batch = y[start:stop]

      derivative = partial_derivative(X_batch, y_batch, m_stat)      
      cumulative_derivative += derivative ** 2
      adjusted_grad = derivative / (fudge_factor + np.sqrt(cumulative_derivative))      

      #updating weight
      m_stat = m_stat - lr*(adjusted_grad+2*reg_para*(m_stat**2))

    if if_print:
      print(f"epoch: {epoch} ----> MSE: {MSE(X, y, m_stat)}")
      
  return m_stat

In [ ]:
# Make submission file
def submission(y_pred):
  predictions = []
  for i in range(len(indices)):
    predictions.append([indices[i], y_pred[i][0]])
  
  # In case of colab's error, I write 2 times for ensuring
  csv_writer = csv.writer(open('109550058.csv', 'w', newline=''))
  csv_writer.writerow(["index", "answer"])
  csv_writer.writerows(predictions)
  csv_writer = csv.writer(open('109550058.csv', 'w', newline=''))
  csv_writer.writerow(["index", "answer"])
  csv_writer.writerows(predictions)

In [ ]:
m_stat = training(X_next, y)

y_pred = (X_test_next @ m_stat) 
# Set y_pred<0 to 0
y_pred[y_pred<0] = 0

submission(y_pred)